In [ ]:
sessionInfo()

In [ ]:
library(scatterplot3d)

# Random number generation

## Uniform random number generation

### Goal

To generate $U_i \stackrel{iid}{\sim} \text{unif}(0, 1)$.

* Basis for all other random number generation
* **Fact: NO RANDOM NUMBER IN COMPUTER** --- only look random (PSEUDO-RANDOM)

### Congruential generator

\begin{align*}
    x_{i+1} &= a x_i \mod m, \quad i=1, 2, \dotsc \\
    u_i &= x_i / m.
\end{align*}

* Modulus $m$ is a large prime number.
* Multiplier $a$ is a positive integer between 2 and $m-1$.
* Map $f: x \mapsto ax \mod m$ maps $\{1, \dotsc, m-1\}$ onto itself and is one-to-one:
    - Suppose $x\in\{1, \dotsc, m-1\}$. Then $f(x) \neq 0$ since $a$ and $m$ are relatively prime hence $ax$ is not a multiple of $m$. Thus $f$ maps $\{1, , \dotsc, m-1\}$ to $\{1, , \dotsc, m-1\}$.
    - If $f(x) = r \in \{1, \dotsc, m-1\}$, then $ax = m k + r$ for some nonnegative integer $k$, hence $x \neq 0$. Hence the map $f$ is onto.
    - If $ay = ax \mod m$, then $a(y - x) = m k$ for some integer $k$. Since $a$ and $m$ are relatively prime,  $y - x = m l$ for some interger $l$. That is, $y = x + p l$ or $x = y \mod m$. Hence map $f$ is one-to-one.
    
* Note
\begin{align*}
    x_1 &= a x_0 \mod m \\
    x_2 &= a x_1 \mod m = a^2 x_0 \mod m \\
    x_3 &= a x_2 \mod m = a^3 x_0 \mod m \\
    & \vdots \\
    x_n &= a x_{n-1} \mod m = a^n x_0 \mod m
\end{align*}
Hence if $a^n = 1 \mod m$ for some $n$, then 
$$
    x_n = x_0 \mod m
$$
and the (pseudo)random number generator repeats the sequence. The number $n$ is called the **period** of the RNG, and $x_0$ is called the **seed**.
    

In [ ]:
set.seed(2020)  # set 2020th seed; does not mean x_0 = 2020
runif(5)
set.seed(2020)  # same seed results in same "random" sequence
runif(5)

* Primitive root of unity
    - Fermat's little theorem: $a^{m} = a \mod m$.
    - Since $a$ is not divisible by $m$, we have $a^{m-1} = 1 \mod m$.
    - Thus the period $n$ satisfies $n \le m - 1$.
    - *Primitive* root of unity: $a$ such that $n = m - 1$
    - If $a$ is primitive, then $x_1, x_2, \dotsc, x_{m-1}$ is a permutation of $\{1, 2, \dotsc, m-1\}$.
    - For $m=2^{31} - 1$ (Mersenne prime), $a=7^5 = 16807$ is primitive, leading to the period of $2^{31} - 2$ = `2,147,483,646`. This RNG was used in early versions of MATLAB (up to v4).
    
Good RNGs should have long periods, and should give samples which appear to be drawn from a uniform distribution. If the size of the sample is much less than the period of the RNG, then the sample should appear random.

### Autocorrelation

* Ideally, a pseudorandom number sequence should look i.i.d. If we take the first $p$ values to form an $p$-dimensional vector, the second $p$ values to form a second $p$-dimensional vector, etc, then these vectors should fill the $p$-dimensional hypercube uniformly.

* However, by construction the sequence generated by congruential generators depends on the previous value, hence tends to be correlated.

* It is known that congruential generators tend to give $p$-dimensional vectors that concentrate lower-dimensional hyperplanes, for some $p$.

In [ ]:
# IBM System/360 RANDU generator
# a = 2^16 + 3 = 65539
# m = 2^31
n <- 2000
a <- 65539
m <- 2^31
u <- vector(length=n)
u[1] <- 1
for (i in seq_len(n-1)) {
    u[i+1] <- a * u[i] %% m
}

In [ ]:
scatterplot3d(u[1:(n-2)], u[2:(n-1)], u[3:n], angle=160)

In [ ]:
# Early MATLAB RNG
# a = 7^5
# m = 2^31 - 1
n <- 2000
a2 <- 7^5
m2 <- 2^31 - 1
v <- vector(length=n)
v[1] <- 1
for (i in seq_len(n-1)) {
    v[i+1] <- a2 * v[i] %% m2
}

In [ ]:
scatterplot3d(v[1:(n-2)], v[2:(n-1)], v[3:n], angle=20)

A simple modification is to introduce *shuffling* in the sequence, which we won't cover in detail.

### R's RNG

* R uses the Mersenne-Twister as the default RNG. This RNG was developed by Matsumoto and Nishimura in 1998, and is the first algorithm whose period ($2^{19937} - 1$) exceeds the number of electron spin changes since the creation of the Universe ($10^{6000}$ against $10^{120}$)!

* Mersenne-Twister guarantees 623 consecutive dimensions to be equidistributed (over the whole period). 

In [ ]:
RNGkind()

## Transformation methods

From now on, we assume the the problem of generating uniform random numbers has been solved for practical purposes.

### Inverse CDF method

For a random variable $X$, let $F$ be its cumulative distribution function (CDF), that is, $F(x) = P(X \le x)$.
Recall that $F$ is right-continuous and nondecreasing. Also, if $F$ is strictrly increasing, random variable $F(X)$ is uniformly distributed on $[0, 1]$. Below, we generalize this result.

The inverse CDF of $X$ is defined as
$$
    F^{-1}(u) = \inf\{x: F(x) \ge u\}, 
    ,
$$
which coincides the usual inverse of $F$ if $F$ is strictly increasing.

**Proposition 1**. Let $X$ be a random variable with CDF $F$. Then the following holds.
1. If $F$ is continuous, then $U = F(X)$ is a uniform random variable on $[0, 1]$.
2. If $F$ is not continous, then $P[F(x) \le y] \le u$ for all $y \in [0, 1]$.
3. If $U$ is uniform on $[0, 1]$, then $F^{-1}(U)$ has CDF $F$.
    
*Proof*. 
* Part 1: We will show that 
$$
    P[F(X) \le F(t)] = F(t)
    \tag{*}
$$
for any $t$. Suppose for now this is true. 

 Let $u \in (0, 1)$. Then by continuity of $F$, there is $y$ such that $F(y) = u$. By (*), 
$$
    P[F(X) \le u] = P[F(X) \le F(t)] = F(t) = u.
$$

* Part 3: It suffices to show that $\{F^{-1}(U) \le t\} = \{U \le F(t)\}$. If $F^{-1}(u)=\inf\{x: F(x) \ge u\} \le t$, then by the monotonicity and right-continuity of $F$, the set $\{x: F(x) \ge u\}$ is an half-closed interval containing its left endpoint, which is $F^{-1}(u)$. Hence $F(F^{-1}(u)) \ge u$. Since $F^{-1}(u) \le t$, again by monotonicity of $F$, it follows that $u \le F(F^{-1}(u)) \le F(t)$. Conversely, if $u \le F(t)$, then by definition $F^{-1}(u) \le t$.

* Part 2: by part 3, $X\stackrel{d}{=}F^{-1}(U)$, where $U$ is uniform on $[0, 1]$. Since $x=F^{-1}(u)$ implies $u \le F(x)$, 
$$
    P[F(X) \le y] \le P(U \le y) = y.
$$

* It remains to show (*). Monotonicity of $F$ yields $\{X > t\} \subset \{F(X) \ge F(t)\}$. Hence $\{X > t \} \cap \{F(X) < F(t) \} = \emptyset$. Likewise $\{X \le t\} \cap \{F(X) > F(t)\} = \emptyset\}$. Then,
\begin{align*}
    \{X > t\} \cap \{F(X) \le F(t)\} &=
    [\{X > t\} \cap \{F(X) < F(t) \} \cup [\{X > t\} \cap \{F(X) = F(t)\}] \\
    &= \{X > t\} \cap \{F(X) = F(t)\}
    \\
    \{X \le t\} \cap \{F(X) \le F(t)\} &= \{X \le t\} \cap \{F(X) > F(t)\}^c 
    = \{X \le t\}
\end{align*}
So,
$$
    \{F(X) < F(t) \}  = \{X \le t\} \cup [\{X > t\} \cap \{F(X) = F(t)\}] =: A \cap B
    .
$$
Obviously events $A$ and $B$ are disjoint. However, event $B$ correspondes to the values $x$ of $X$ with which $F(x)$ is constant. Hence $P(B)=0$. Therefore,
$$
    P[F(X) < F(t)] = P(A) + P(B) = P(A) = P(X \le t) = F(t).
$$

#### Exponential distribution

* CDF $F(x) = 1 - e^{-\lambda x}$ yields $F^{-1}(u) = -\frac{1}{\lambda}\log u$ on $(0, 1)$. 

In [ ]:
# Exp(1)
n <- 1000
u <- runif(n)
x <- -log(u)
hist(x)

#### Cauchy distribution

From the density of the Cauchy distribution
$$
    f(x) = \frac{\beta}{\pi(\beta^2 + (x-\mu)^2)},
$$
its CDF is $F(x) = \frac{1}{2} + \frac{1}{\pi}\tan^{-1}\left(\frac{x-\mu}{\beta}\right)$. Thus
$$
    F^{-1}(u) = \mu + \beta\tan(\pi u - \pi/2) = \mu - \frac{\beta}{\tan(\pi u)}
    .
$$

In [ ]:
# standard Cauchy (beta=1, mu=0)
n <- 1000
u <- runif(n)
x <- -1/tan(pi * u)
hist(x, breaks=40)

#### Discrete uniform distribution 

$X \sim \text{unif}(\{1, 2, \dotsc, k\})$. It is easy to verify $F(x) = \frac{1}{k}\lfloor x \rfloor$ for $x \in [0, n]$ and $F^{-1}(u)=\lceil ku \rceil$.


In [ ]:
k <- 10
n <- 1000
u <- runif(n)
x <- ceiling(k * u)
table(x)

#### Geometric distribution 

If $X \sim \text{geom}(p)$, then its probability mass functin $p(x) = (1-p)^{x-1}p$.

For $Y \sim \text{Exp}(\lambda)$, 
\begin{align*}
    P(\lfloor Y \rfloor = k) &= P(k-1 < Y \le k) = F_Y(k) - F_Y(k-1) = (1 - e^{-\lambda k}) - (1 - e^{-\lambda(k-1)}) \\
   &=  e^{-\lambda(k-1)}(1 - e^{-]lambda}) \\
   &- (1 - p)^{k-1} p
\end{align*}
if $\lambda$ satisfies $p = 1 - e^{-\lambda}$, or $\lambda = -\log(1-p)$.

For this $\lambda$, $X = \lceil Y \rceil = \lceil -\frac{1}{\lambda}\log U \rceil = \lceil \frac{\log U}{\log(1-p)}\rceil$.

In [ ]:
p <- 0.3
n <- 1000
u <- runif(n)
y <- log(u) / log(1 - p)
x <- ceiling(y)
hist(x)

### Normal random numbers

For $X \sim N(0, 1)$, inverse CDF $\Phi^{-1}$ does not have a closed form.

#### Box-Muller

Generates $X, Y \stackrel{iid}{\sim} N(0, 1)$.

Transforms the random Cartesian coordinates $(X, Y)$ to polar coorinates $(R, \Theta)$. Since $(X, Y)=(R\cos\Theta, R\sin\Theta)$,
$$
    \iint f_{XY}(x, y)dxdy = \frac{1}{2\pi}\exp(-\frac{x^2 + y^2}{2})dxdy
    = \iint \frac{1}{2\pi}\exp(-\frac{r^2}{2})rdrd\theta
    .
$$

Hence $R$ has desnity $f_R(r) = r\exp(-\frac{r^2}{2})$ and $\Theta$ is uninform on $[0, 2\pi]$. Since
$$
    P(R > \rho) = P(R^2 > \rho^2) = \int_\rho^{\infty} r\exp(-\frac{r^2}{2})dr = \exp(-\frac{1}{2}r^2),
$$
random variable $R^2$ is exponentially distributed with $\lambda = 1/2$.

Thus for independent $U, V \sim \text{unif}(0, 1)$, set
$$
    R = (-2\log U)^{1/2}, \quad \Theta = 2\pi V
    .
$$
Then $(X, Y) = (R\cos\Theta, R\sin\Theta)$ are independently $N(0, 1)$.

In [ ]:
n <- 500
u <- runif(n)
v <- runif(n)
r <- sqrt(-2 * log(u))
theta <- 2 * pi * v
x <- r * cos(theta)
y <- r * sin(theta)
hist(c(x, y))

#### Marsaglia

* From the Box-Muller transformation, we learned that if $R^2 \sim \text{Exp}(1/2)$ and $\Theta\sim\text{unif}(0, 2\pi)$ and they are indepdenent, $(R\cos\Theta, R\sin\Theta)$ are i.i.d. standard normal. 

* Let $F(r) = 1 - \exp(-r^2/2)$ be the CDF of $R$. Then $S = F(R)$ is uniform on $(0, 1)$, so is $T = 1 - S$. Then, $(U, V) = (T\cos\Theta, T\sin\Theta)$ is *uniformly* distributed in the unit disk.

* Conversely, if we sample $(U, V)$ uniformly from the unit disk, and set $T = \sqrt{U^2 + V^2}$, $\cos\Phi = U/T$, $\sin\Phi=V/T$, then $-2\log T$ is identically distributed as $R^2$, i.e., $\text{Exp}(1/2)$, and $\Phi$ is identically distributed as $\Theta$, i.e., uniform on $(0, 2\pi)$. Therefore, if we set $(X, Y)$ such that
$$
    X = \sqrt{-2\log T}\frac{U}{T},
    \quad
    Y = \sqrt{-2\log T}\frac{V}{T},
$$
then $X, Y$ are i.i.d. standard normal.

* One way to sample from the unit disk is to sample $(U, V)$ from $\text{unif}[-1, 1]\times\text{unif}[-1, 1]$, and discard the sample if $U^2 + V^2 > 1$ and resample.

* Algorithm:
    1. $U, V \stackrel{iid}{\sim} \text{unif}[-1, 1]$;
    2. If $T = \sqrt{U^2 + V^2} > 1$, go to 1;
    3. Set $X = \sqrt{-2\log T}\frac{U}{T}$ and $Y = \sqrt{-2\log T}\frac{V}{T}$.
    
* This method avoids the trigonometric function evaluations of the Box-Muller, but uses $4/\pi$ as many random pairs on average.

In [ ]:
n <- 500
it <- 0
x <- numeric(n)
y <- numeric(n)
while (it < n) {
    u <- 2 * runif(1) - 1
    v <- 2 * runif(1) - 1
    tau <- sqrt(u^2 + v^2)
    if (tau > 1) next
    x[it] <- sqrt(-2 * log(tau)) * u / tau
    y[it] <- sqrt(-2 * log(tau)) * v / tau
    it <- it + 1
}
hist(c(x, y))